# <오늘 할 것: Socket 이모저모>

# 1. Web 기초
- 1-1. OSI 참조 모델

# 2. Socket
- 2-1. 서버
    - 2-1-1. 최고로 간단하게 만들어보기
    - 2-1-2. Post 방식으로 해보기
    - 2-1-3. 클라이언트에게 데이터 보내기
    - 2-1-4. 계속 접속받기
        - 2-1-4-1. 현재 시각을 띄우는 서버
    - 2-1-5. 클라이언트가 요청한 데이터를 전송하는 서버
    - 2-1-6. 이미지를 띄워보자
    - 2-1-7. 파이썬 실행파일을 띄워보자
    - 2-1-8. Multi thread로 실행하기
- 2-2. 클라이언트
### - 2-3. Python + HTML


# 1. Web 기초

## 1-1. OSI 참조 모델
 > Open System Interconnection
 
 > 7개 계층으로 web 구조를 표준화한 모델
- 사용자와 접속
    - 응용
    - 표현
    - 세션
    - 전송
- 망과 접속
    - 네트워크
    - 데이터링크
    - 물리

<img src=osi.jpg>

- 응용, 전송 계층만 제대로 알고 있으면 된다.

### - 전송계층
> 프로토콜(TCP, UDP)과 관련된 계층으로 오류 복구와 흐름 제어 등을 담당하며, 두 시스템 간에 신뢰성 있는 데이터를 전송한다.
    - TCP: 등기; 주고 받음을 확인할 수 있다; 대부분 TCP
    - UDP: 우편함; 주고 받음을 확인할 수 없다; 고속이다


### - 응용계층
> 사용자와 직접적으로 상호작용하는 계층이다. 구글 크롬(Google Chrome), 파이어폭스(Firefox), 사파리(Safari) 등 웹 브라우저와 스카이프(Skype), 아웃룩(Outlook), 오피스(Office) 등의 응용 프로그램이 대표적이다.

출처:
http://www.ciokorea.com/news/36536#csidx67f960ca665aac8b8e30fde3e558ed4 
    


### - IP
> 개별 컴퓨터를 구별하기 위한 고유 주소
    - 4자리의 숫자로 구성되어 있다.
        - 000.000.000.000 ~ 255.255.255.255



### - 포트 번호
> 통신을 할 구체적인 주소
    - 16비트로 이루어져 있다.
        - 0 ~ 65536



# 2. Socket

> 소켓 프로그래밍에 필요한 시스템 콜을 래핑하는 API를 제공하는 모듈이다.
- 소켓 프로그래밍이란?: 네트워크 프로그래밍 분야에서 소켓은 연결된 네트워크의 양 끝단을 추상화 시킨 개념이며, 컴퓨터의 관점에서는 네트워크로 통하는 컴퓨터의 외부와 컴퓨터 내부의 프로그램을 이어주는 인터페이스이다. 네트워크를 통해서 바이트스트림을 주고 받을 수 있는 창구라 보면 된다. 다만 단순히 프로그램의 내부와 외부를 잇는 표준 입출력과는 달리 소켓은 네트워크의 반대편이 어디인지에 대한 정보를 가지고 있다. 즉 우리가 택배를 보낼 때 박스에 물건을 넣고 받는 사람 주소를 쓰는 것과 비슷하게 소켓은 어디로 보내지는 창구라는 것이 명시된 택배 상자 같은 것이다.

출처: https://soooprmx.com/archives/8737
- 엄청 자세히 설명되어 있다!

## 2-1. 서버
- 0211_Server.ipynb 참조

## 2-2. 클라이언트
- 0211_Client.ipynb 참조

## 2-3. Python + HTML

생각보다 간단하게 템플릿을 구현할 수 있다.

1. HTML과 Python 파일을 물리적으로 분리한다.
    - e.g. HTML은 전부 위에, Python은 전부 아래에
    

2. HTML에서 동적 처리가 필요한 부분에는 마크를 한다.
    - e.g. 변수처리되어야 할 것에 `@` 붙여두기
    

3. `replace` 함수를 활용해서 `@`을 달고 있는 것들을 바꿔준다.

In [ ]:
html = """

<html>
   <head>
      <meta charset="utf-8">
   </head>

   <body>
      <font color=red> @out </font>
   </body>
</html>
"""

html = html.replace("@out", "제목출력")

print(html)

In [2]:
import socket
import os.path
import subprocess

In [19]:
img_ext = ['.jpg', '.png', '.jpeg', '.bmp', '.ico']
text_ext = ['.html', '.txt']
py_ext = ['.py']

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind(('localhost', 85))
server_socket.listen(0)
print('listening')

if True:
    client_socket, addr = server_socket.accept()
    print('accepting')
    data = client_socket.recv(65535)
    #print(data.decode())
    
    try:
        headers = data.decode().split('\r\n')
        filename = headers[0].split(' ')[1]

        ext = os.path.splitext(filename)[1]

        if ext in text_ext:   #텍스트일때
            file = open("." + filename, 'rt', encoding = 'utf-8')
            html = file.read()
            header = 'HTTP/1.0 200 OK\r\n'
            client_socket.send(header.encode('utf-8'))
            client_socket.send('Content-Type: text/html\r\n\r\n'.encode())  #html임을 명시
            client_socket.send(html.encode('utf-8'))
        
        elif ext in py_ext:
            header = 'HTTP/1.0 200 OK\r\n'
            output = subprocess.check_output(['python.exe', "." + filename])
            html = output.decode('cp949')
            client_socket.send(header.encode('utf-8'))
            client_socket.send('Content-Type: text/html\r\n\r\n'.encode()) #html임을 명시
            client_socket.send(html.encode('utf-8'))  #한글이 포함되어있을 때는 'cp949'!!!!
        
        elif ext in img_ext:   #이미지일 때
            header = 'HTTP/1.1 200 OK\r\n'
            client_socket.send(header.encode())
            client_socket.send('Content-Type: image/jpg\r\n'.encode())  #이미지임을 명시
            client_socket.send('Accept-Ranges: bytes\r\n\r\n'.encode())  #이미지 크기(몰라도 됨)
            file = open("." + filename, 'rb')  #rb: 이미지니까 바이너리로 읽어온다.
            client_socket.send(file.read())  #이미지는 이미 byte 형태라서 인코딩 필요x
            file.close()

        else:    #아무것도 아닐 때
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'
            client_socket.send(header.encode('utf-8'))
    except Exception as e:
        print(e)
    client_socket.close()

listening
accepting


`replace`를 써서 바꾸는 것도 잘 되기는 하는데,

**딕셔너리**를 활용하면 더 간편하다.

In [17]:
def render(html, data):
    for key in data:
        html = html.replace('@' + key, data[key])
    return html

In [11]:
html = 'hello @v1   test @v2    item @v3 '
data = {'v1': '안녕하세요', 'v2': '이순신', 'v3': '^^'}
html = render(html, data)
print(html)

hello 안녕하세요   test 이순신    item ^^ 


In [12]:
#html이 별도의 파일로 있는 경우
def renderfile(file, data):
    html = open(file, 'rt', encoding = 'utf-8').read()
    for key in data:
        html = html.replace('@' + key, data[key])
    return html

In [20]:
data = {"title": "나의 홈페이지", "name": "이순신", "email": "^^"}

html = renderfile("template.txt", data)

print(html)

<html>
   <head>
      <meta charset="utf-8">
   </head>

   <body>
      <font color=red> 나의 홈페이지 </font>
      <table border=1>
         <tr>
            <td> 이름 </td>   <td> 이순신 </td>
        </tr>
        <tr>
           <td> 이메일 </td> <td> ^^ </td>
       </tr>
   </body>
</html>


#### HTML 모듈을 활용하면 굳이 서버에 띄우지 않고도 html을 검증할 수 있다.

In [21]:
from IPython.core.display import HTML

In [22]:
HTML(html)

이름,이순신
이메일,^^
